In [4]:
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import tensorflow as tf

# tf.reset_default_graph()

x_data = [[0., 0.],[0., 1.],[1., 0.],[1., 1.]]
y_data = [[0.],[1.],[1.],[0.]]

model = Sequential()
model.add(Dense(5, input_dim=2, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])
model.summary()
model.fit(x_data, y_data, epochs=500)

score = model.evaluate(x_data, y_data, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

print model.predict(x_data)

show_graph(tf.get_default_graph().as_graph_def())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             (None, 5)                 15        
_________________________________________________________________
dense_58 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_59 (Dense)             (None, 1)                 6         
Total params: 51.0
Trainable params: 51
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/500
4/4 [==============================] - 0s - loss: 0.5004 - acc: 0.5000
Epoch 2/500
4/4 [==============================] - 0s - loss: 0.4861 - acc: 0.5000
Epoch 3/500
4/4 [==============================] - 0s - loss: 0.4735 - acc: 0.5000
Epoch 4/500
4/4 [==============================] - 0s - loss: 0.4625 - acc: 0.5000
Epoch 5/500
4/4 [==============================] - 0s - loss: 0.4527 - acc